<a href="https://colab.research.google.com/github/Tejaswi-kashyap-006/NLP_Projects/blob/main/sentiment_analysis_with__BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 735.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    Uninstalling torchvision-0.14.1+cu116:
      Successfully uninstalled torchvision-0.14.1+cu116
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.13.1+cu116
    Uninstalling torchaudio-0.13.1+cu116:
      Successfully uninstalled torchaudio-0.13.1+cu116
ERROR: pip's dependency resolver does not

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.6 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
#instantiate model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
#encode and calculate sentiment 

In [22]:
tokens = tokenizer.encode('I loved this, absolutely the best', return_tensors='pt')

In [23]:
result = model(tokens)

In [24]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.3882, -2.5314, -1.2469,  1.1947,  4.1058]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

In [25]:
result.logits

tensor([[-2.3882, -2.5314, -1.2469,  1.1947,  4.1058]],
       grad_fn=<AddmmBackward>)

In [26]:
int(torch.argmax(result.logits))+1

5

In [27]:
#collect review 


In [29]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [31]:
reviews[0]

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'

In [32]:
import pandas as pd
import numpy as np


In [33]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [36]:
df.tail()

,review
6,We came for brunch twice in our week-long visi...
7,I went here a little while ago- a beautiful mo...
8,Great coffee and vibe. That's all you need. C...
9,Great coffee and vibe. That's all you need. C...
10,Good coffee and toasts. Straight up and down -...


In [41]:
df['review'].iloc[0]

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'

In [37]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result =model(tokens)   
    return int(torch.argmax(result.logits))+1

In [39]:
sentiment_score(df['review'].iloc[1])

3

In [42]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [43]:
df

,review,sentiment
0,Great food amazing coffee and tea. Short walk ...,5
1,It was ok. Had coffee with my friends. I'm new...,3
2,Great staff and food. Must try is the pan fri...,5
3,Ricotta hot cakes! These were so yummy. I ate ...,5
4,I came to Social brew cafe for brunch while ex...,5
5,It was ok. The coffee wasn't the best but it w...,3
6,We came for brunch twice in our week-long visi...,4
7,I went here a little while ago- a beautiful mo...,2
8,Great coffee and vibe. That's all you need. C...,5
9,Great coffee and vibe. That's all you need. C...,4


In [45]:
df['review'].iloc[5]

"It was ok. The coffee wasn't the best but it was fine. The relish on the breakfast roll was yum which did make it sing. So perhaps I just got a bad coffee but the food was good on my visit."